In [1]:
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import cv2
from glob import glob
import shutil
from tqdm import tqdm

In [2]:
# 高斯曲线函数
def gauss_curve(src, mean, sigma):
    dst = np.exp(-(src - mean)**2 / (2 * sigma**2))
    return dst

# 曝光融合类
class ExposureFusion(object):
    def __init__(self, sequence, best_exposedness=0.5, sigma=0.2, eps=1e-12, exponents=(1.0, 1.0, 1.0), layers=7):  # 初始化
        """
        sequence: 输入图像序列，形状为 [N, H, W, 3]，像素值范围在 (0, 1) 之间，dtype=float32
        best_exposedness: 最佳曝光度参数，默认为 0.5
        sigma: 高斯曲线的标准差参数，默认为 0.2
        eps: 避免除零的小常数，默认为 1e-12
        exponents: 对比度、饱和度和曝光度的指数参数，默认为 (1.0, 1.0, 1.0)
        layers: 多分辨率融合中金字塔层数，默认为 7
        """
        self.sequence = sequence  # 输入图像序列
        self.img_num = sequence.shape[0]  # 图像数量
        self.best_exposedness = best_exposedness  # 最佳曝光度参数
        self.sigma = sigma  # 高斯曲线的标准差参数
        self.eps = eps  # 避免除零的小常数
        self.exponents = exponents  # 对比度、饱和度和曝光度的指数参数
        self.layers = layers  # 多分辨率融合中金字塔层数

    @staticmethod
    def cal_contrast(src):  # 计算对比度
        gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        laplace_kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=np.float32)
        contrast = cv2.filter2D(gray, -1, laplace_kernel, borderType=cv2.BORDER_REPLICATE)
        return np.abs(contrast)

    @staticmethod
    def cal_saturation(src):  # 计算饱和度
        mean = np.mean(src, axis=-1)
        channels = [(src[:, :, c] - mean)**2 for c in range(3)]
        saturation = np.sqrt(np.mean(channels, axis=0))
        return saturation

    @staticmethod
    def cal_exposedness(src, best_exposedness, sigma):  # 计算曝光度
        exposedness = [gauss_curve(src[:, :, c], best_exposedness, sigma) for c in range(3)]
        exposedness = np.prod(exposedness, axis=0)
        return exposedness

    def cal_weight_map(self):  # 计算权重图
        weights = []
        for idx in range(self.sequence.shape[0]):
            contrast = self.cal_contrast(self.sequence[idx])
            saturation = self.cal_saturation(self.sequence[idx])
            exposedness = self.cal_exposedness(self.sequence[idx], self.best_exposedness, self.sigma)
            weight = np.power(contrast, self.exponents[0]) * np.power(saturation, self.exponents[1]) * np.power(exposedness, self.exponents[2])
            # Gauss Blur
            # weight = cv2.GaussianBlur(weight, (21, 21), 2.1)
            weights.append(weight)
        weights = np.stack(weights, 0) + self.eps
        # normalize
        weights = weights / np.expand_dims(np.sum(weights, axis=0), axis=0)
        return weights

    def naive_fusion(self):  # 加权平均融合结果图像
        weights = self.cal_weight_map()  # 计算权重地图 [N, H, W]
        weights = np.stack([weights, weights, weights], axis=-1)  # [N, H, W, 3]
        naive_fusion = np.sum(weights * self.sequence * 255, axis=0)
        naive_fusion = np.clip(naive_fusion, 0, 255).astype(np.uint8)
        return naive_fusion

    def build_gaussian_pyramid(self, high_res):  # 构建高斯金字塔
        gaussian_pyramid = [high_res]
        for idx in range(1, self.layers):
            gaussian_pyramid.append(cv2.GaussianBlur(gaussian_pyramid[-1], (5, 5), 0.83)[::2, ::2])
        return gaussian_pyramid

    def build_laplace_pyramid(self, gaussian_pyramid):  # 构建拉普拉斯金字塔
        laplace_pyramid = [gaussian_pyramid[-1]]
        for idx in range(1, self.layers):
            size = (gaussian_pyramid[self.layers - idx - 1].shape[1], gaussian_pyramid[self.layers - idx - 1].shape[0])
            upsampled = cv2.resize(gaussian_pyramid[self.layers - idx], size, interpolation=cv2.INTER_LINEAR)
            laplace_pyramid.append(gaussian_pyramid[self.layers - idx - 1] - upsampled)
        laplace_pyramid.reverse()
        return laplace_pyramid

    def multi_resolution_fusion(self):  # 融合图像
        weights = self.cal_weight_map()  # 计算权重图 [N, H, W]
        weights = np.stack([weights, weights, weights], axis=-1)  # [N, H, W, 3]

        image_gaussian_pyramid = [self.build_gaussian_pyramid(self.sequence[i] * 255) for i in range(self.img_num)]
        image_laplace_pyramid = [self.build_laplace_pyramid(image_gaussian_pyramid[i]) for i in range(self.img_num)]
        weights_gaussian_pyramid = [self.build_gaussian_pyramid(weights[i]) for i in range(self.img_num)]

        fused_laplace_pyramid = [np.sum([image_laplace_pyramid[n][l] *
                                         weights_gaussian_pyramid[n][l] for n in range(self.img_num)], axis=0) for l in range(self.layers)]

        result = fused_laplace_pyramid[-1]
        for k in range(1, self.layers):
            size = (fused_laplace_pyramid[self.layers - k - 1].shape[1], fused_laplace_pyramid[self.layers - k - 1].shape[0])
            upsampled = cv2.resize(result, size, interpolation=cv2.INTER_LINEAR)
            result = upsampled + fused_laplace_pyramid[self.layers - k - 1]
        result = np.clip(result, 0, 255).astype(np.uint8)

        return result


In [3]:
def exposure_fusion(root_path):
    sequence_path = [os.path.join(root_path, train) for train in os.listdir(root_path)]
    sequence = np.stack([cv2.imread(path) for path in sequence_path], axis=0)
    mef = ExposureFusion(sequence.astype(np.float32) / 255.0)
    naive_fusion_result = mef.naive_fusion()
    multi_res_fusion = mef.multi_resolution_fusion()
    return multi_res_fusion

In [4]:
def get_subfolders(directory):
    subfolders = [os.path.join(directory, name) for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
    return subfolders

In [5]:
# 复制原图
def get_original_image(root_path, output_path):
    sequence_path = [os.path.join(root_path, train) for train in os.listdir(root_path)]
    for path in sequence_path:
        pattern = os.path.join(root_path, '**', '*_1.0.jpg')  # 原图
        matching_paths=glob(pattern, recursive=True)
        for image_path in matching_paths:
            file_name = os.path.basename(os.path.dirname(image_path))
            parent_path = os.path.join(output_path, file_name)
            os.makedirs(parent_path, exist_ok=True)
            new_file_path = os.path.join(parent_path, file_name+"_0.jpg")  # 0-原图，1-方法1 ，2-方法2
            shutil.copyfile(image_path, new_file_path)

In [13]:
# 对输入路径子文件夹中的图片进行HDR合成并按规则存储
def get_exposure_fusion_results(root_path, output_path):
    subfolders = get_subfolders(root_path)
    image_list = []
    for path in tqdm(subfolders):
        new_file_path = os.path.join(output_path, os.path.basename(path))
        os.makedirs(new_file_path, exist_ok=True)
        new_file_path = os.path.join(new_file_path, os.path.basename(path)+"_1.jpg")  # 0-原图，1-方法1 ，2-方法2
        cv2.imwrite(new_file_path, exposure_fusion(path))

In [15]:
# 对测试集随机抽取图片进行HDR合成
root_path = 'myOutput'
output_path = 'result_test'
get_original_image(root_path, output_path)
get_exposure_fusion_results(root_path, output_path)  

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:54<00:00,  1.75s/it]


In [16]:
# 对作业1中三种图片选取曝光程度居中的一张图片使用模型生成多曝光图片后进行HDR合成
root_path = 'output'
output_path = 'result'
get_original_image(root_path, output_path)
get_exposure_fusion_results(root_path, output_path)  

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:44<00:00, 11.25s/it]
